In [9]:
import os
from dotenv import load_dotenv

In [14]:
# Load environment variables from a .env file
load_dotenv()

video_url = 'https://www.youtube.com/watch?v=OPb-nj9-Qsc&t=1189s'
print(video_url)

https://www.youtube.com/watch?v=OPb-nj9-Qsc&t=1189s


In [15]:
from googleapiclient.discovery import build
import re

# Replace with your actual API key
googleKey = os.getenv('googleKey')

# Create a YouTube API client
youtube = build('youtube', 'v3', developerKey=googleKey)

def remove_time_tag(url):
    # Remove time tag if present
    return re.sub(r'[&?]t=\d+[hms]?(&?)', r'\1', url)

def get_video_info(url):
    # Remove time tag
    clean_url = remove_time_tag(url)
    
    # Extract video ID
    video_id_match = re.search(r'(?:v=|\/)([0-9A-Za-z_-]{11}).*', clean_url)
    if not video_id_match:
        return None, None
    
    video_id = video_id_match.group(1)
    
    # Get video details
    try:
        response = youtube.videos().list(
            part='snippet',
            id=video_id
        ).execute()
        
        if not response['items']:
            return None, None
        
        channel_id = response['items'][0]['snippet']['channelId']
        channel_title = response['items'][0]['snippet']['channelTitle']
        
        return channel_title, channel_id
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

# Example usage
#url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ&t=1m30s"
channel_name, channel_id = get_video_info(video_url)

if channel_name and channel_id:
    print(f"Channel Name: {channel_name}")
    print(f"Channel ID: {channel_id}")
else:
    print("Could not retrieve channel information.")


Channel Name: 亂gad - randgad
Channel ID: UCFPAwIW39LR4JwwXFr8DhpQ


In [19]:

def get_channel_videos(channel_id):
    videos = []
    next_page_token = None
    
    while True:
        # Request channel uploads playlist
        res = youtube.channels().list(id=channel_id, 
                                      part='contentDetails').execute()
        playlist_id = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']
        
        # Request playlist items
        res = youtube.playlistItems().list(playlistId=playlist_id,
                                           part='snippet',
                                           maxResults=10,
                                           pageToken=next_page_token).execute()
        
        # Process video items
        for item in res['items']:
            video_id = item['snippet']['resourceId']['videoId']
            title = item['snippet']['title']
            publishAt = item['snippet']['publishedAt']
            videos.append({'id': video_id, 'title': title, 'publishAt': publishAt})
        
        # Check for next page
        next_page_token = res.get('nextPageToken')
        if not next_page_token:
            break
    
    return videos


video_list = get_channel_videos(channel_id)

# Print the list of videos
for video in video_list:
    print(f"Video ID: {video['id']}, Publish At: {video['publishAt']}, Title: {video['title']}")


Video ID: OPb-nj9-Qsc, Publish At: 2025-01-06T05:13:00Z, Title: 亂‌‌‌gad‌‌‌ ‌‌‌‌‌第745集 - 阿Ed 多倫多之旅 / Switch 2 / Starship IFT 7 / Deepseek v3 / AI 影片 Sora 終於有得試
Video ID: PTBMjuWgSRA, Publish At: 2025-01-04T17:02:18Z, Title: 亂‌‌‌gad‌‌‌ ‌‌‌‌‌第745集 - 阿Ed 多倫多之旅 / Switch 2 / Starship IFT 7 / Deepseek v3 / AI 影片 Sora 終於有得試
Video ID: iUm0rBbHfSI, Publish At: 2024-12-28T17:32:12Z, Title: 亂‌‌‌gad‌‌‌ ‌‌‌‌‌第744集 - Excel 電競 MEWC 2024 / OpenAI 轉型做盈利公司 / $19 藍牙耳機有冇伏
Video ID: vPs13eLXsI0, Publish At: 2024-12-28T08:40:00Z, Title: 亂‌‌‌gad‌‌‌ ‌‌‌‌‌第‌‌‌744集 ~ Excel 電競 MEWC 2024 / OpenAI 轉型做盈利公司 / HK$19 藍牙耳機有冇伏
Video ID: 6ptd62EOPg8, Publish At: 2024-12-21T17:43:00Z, Title: 亂‌‌‌gad‌‌‌ ‌‌‌‌‌第743集 - Apple Broadcom AI 晶片 / TP-Link 可能被美國禁用 / Lenovo Legion Go S / OpenAI 打電話玩 AI
Video ID: pbJLNBPEOPg, Publish At: 2024-12-21T16:46:53Z, Title: 亂‌‌‌gad‌‌‌ ‌‌‌‌‌第743集 - Apple Broadcom AI 晶片 / TP-Link 可能被美國禁用 / Lenovo Legion Go S / OpenAI 打電話玩 AI
Video ID: GMUGD1Vnc4w, Publish At: 2024-12-15T07:13:00Z, Title: 亂‌‌‌gad